# Agent-Workshop Examples

This notebook demonstrates the pre-built agents available in agent-workshop v0.2.0:
- **Standalone Agents**: Single-purpose validators (80% use case)
- **LangGraph Workflows**: Multi-step validation pipelines (15% use case)

## Installation

```bash
# Install agent-workshop with all pre-built agents
pip install 'agent-workshop[agents]'
# or using uv:
uv add 'agent-workshop[agents]'
```

## Environment Setup

Create a `.env.development` file with your Claude SDK credentials:

```bash
# .env.development
ENVIRONMENT=development
CLAUDE_API_KEY=your_claude_sdk_key_here

# Optional: Langfuse for observability (can be disabled)
LANGFUSE_ENABLED=false
# LANGFUSE_PUBLIC_KEY=your_public_key
# LANGFUSE_SECRET_KEY=your_secret_key
```

In [ ]:
# Import required libraries
from agent_workshop import Config
from agent_workshop.agents.validators import DeliverableValidator
from agent_workshop.agents.validators.presets import get_preset, list_presets
from agent_workshop.agents.pipelines import ValidationPipeline
import asyncio

## Example Content

We'll use this sample financial report for all examples:

In [ ]:
SAMPLE_FINANCIAL_REPORT = """
# Q4 2024 Financial Report

## Executive Summary
Revenue increased 15% YoY to $10M. Net income reached $3M with strong cash flow.

## Financial Data
- Revenue: $10M (source: accounting system, verified 2024-12-31)
- Expenses: $7M
- Net income: $3M
- Cash on hand: $5M

## Key Metrics
- Gross margin: 30%
- Operating margin: 25%
- EBITDA: $3.5M

## Regulatory Compliance
All reporting follows GAAP standards. SOX compliance confirmed by external audit.

## Risk Factors
- Market volatility may impact Q1 2025 revenue
- Supply chain disruptions remain a concern
"""

---

# Part 1: Standalone Agents

Standalone agents are simple, single-purpose validators that extend the base `Agent` class.
They perform **one task per execution** with **one LLM call**.

## Example 1: Using Built-in Presets

Agent-workshop comes with 6 domain-specific presets that are production-ready.

In [ ]:
# List all available presets
presets = list_presets()
print("Available presets:")
for preset in presets:
    print(f"  - {preset}")

In [ ]:
# Example 1A: Financial Report Validation with Preset
async def example_1a_preset_validator():
    """
    Use the financial_report preset for GAAP/SEC compliance validation.
    
    This preset includes:
    - Financial compliance-focused system prompt
    - 6 validation criteria (data sourcing, GAAP compliance, risk disclosure, etc.)
    - JSON output format
    """
    print("\n" + "="*70)
    print("Example 1A: Financial Report Validator (Preset)")
    print("="*70)
    
    # Get the financial_report preset
    preset = get_preset("financial_report")
    
    # Create validator with preset configuration
    config = Config()
    validator = DeliverableValidator(config, **preset)
    
    print(f"\nValidator configured with:")
    print(f"  - {len(validator.validation_criteria)} validation criteria")
    print(f"  - Output format: {validator.output_format}")
    
    # Run validation
    print(f"\nRunning validation...")
    result = await validator.run(SAMPLE_FINANCIAL_REPORT)
    
    print(f"\n✓ Validation complete!")
    print(f"  - Timestamp: {result['timestamp']}")
    print(f"\nValidation Result (first 500 chars):")
    print(result['validation'][:500] + "...")
    
    return result

# Run the example
result_1a = await example_1a_preset_validator()

## Example 2: Programmatic Configuration

You can create validators with custom prompts and criteria programmatically.

In [ ]:
# Example 2: Custom Validator with Programmatic Config
async def example_2_programmatic_config():
    """
    Create a validator with custom prompts and criteria.
    
    This approach gives you full control over:
    - System prompt (agent's role and expertise)
    - Validation criteria (what to check)
    - User prompt template (how to format the request)
    - Output format (json, detailed, or summary)
    """
    print("\n" + "="*70)
    print("Example 2: Custom Validator (Programmatic Config)")
    print("="*70)
    
    # Create validator with custom configuration
    validator = DeliverableValidator(
        config=Config(),
        system_prompt="You are a financial compliance validator focused on accuracy and completeness.",
        validation_criteria=[
            "All numbers are sourced and verified",
            "GAAP compliance is confirmed",
            "Risk factors are disclosed"
        ],
        output_format="json"
    )
    
    print(f"\nValidator configured with:")
    print(f"  - Custom system prompt")
    print(f"  - {len(validator.validation_criteria)} custom criteria")
    print(f"  - Output format: {validator.output_format}")
    
    # Run validation
    print(f"\nRunning validation...")
    result = await validator.run(SAMPLE_FINANCIAL_REPORT)
    
    print(f"\n✓ Validation complete!")
    print(f"\nValidation Result (first 500 chars):")
    print(result['validation'][:500] + "...")
    
    return result

# Run the example
result_2 = await example_2_programmatic_config()

## Example 3: YAML Configuration

Store your prompts and criteria in a YAML file for easy reuse and version control.

In [ ]:
# First, create a sample prompts.yaml file
yaml_config = """
deliverable_validator:
  system_prompt: |
    You are a financial compliance expert specializing in quarterly reports.
    Focus on accuracy, completeness, and regulatory compliance.
  
  validation_criteria:
    - Executive summary is clear and concise
    - All financial data includes sources
    - Calculations are accurate
    - GAAP compliance is confirmed
    - Risk factors are disclosed
  
  output_format: detailed
"""

# Write the YAML file
with open('example_prompts.yaml', 'w') as f:
    f.write(yaml_config)

print("Created example_prompts.yaml")

In [ ]:
# Example 3: Validator with YAML Configuration
async def example_3_yaml_config():
    """
    Load validator configuration from YAML file.
    
    Benefits:
    - Easy to version control
    - Share configurations across team
    - No code changes needed to update prompts
    """
    print("\n" + "="*70)
    print("Example 3: Validator with YAML Config")
    print("="*70)
    
    # Create validator that loads from YAML file
    validator = DeliverableValidator(
        config=Config(),
        config_file="example_prompts.yaml"
    )
    
    print(f"\nValidator loaded from YAML with:")
    print(f"  - {len(validator.validation_criteria)} criteria from file")
    print(f"  - Output format: {validator.output_format}")
    
    # Run validation
    print(f"\nRunning validation...")
    result = await validator.run(SAMPLE_FINANCIAL_REPORT)
    
    print(f"\n✓ Validation complete!")
    print(f"\nValidation Result (first 500 chars):")
    print(result['validation'][:500] + "...")
    
    return result

# Run the example
result_3 = await example_3_yaml_config()

## Example 4: Other Domain Presets

Let's try the research_paper preset to show how versatile the validators are.

In [ ]:
# Sample research paper abstract for testing
SAMPLE_RESEARCH_PAPER = """
# The Impact of Machine Learning on Financial Forecasting

## Abstract
This study examines the effectiveness of machine learning models in predicting stock market trends.
We compare traditional econometric models with modern deep learning approaches.

## Introduction
Financial forecasting has evolved significantly with the advent of machine learning.
Previous studies (Smith et al., 2020) have shown promising results with neural networks.

## Methodology
We collected 10 years of historical stock data from the S&P 500.
Models tested: LSTM, Random Forest, and ARIMA.

## Results
LSTM models showed 15% improvement in prediction accuracy compared to ARIMA.
Statistical significance: p < 0.01

## Discussion
Our findings suggest that deep learning models outperform traditional methods.
However, computational costs and interpretability remain challenges.

## References
Smith, J. et al. (2020). Neural Networks in Finance. Journal of Finance, 75(3), 234-256.
"""

In [ ]:
# Example 4: Research Paper Validation
async def example_4_research_paper():
    """
    Use the research_paper preset for academic paper validation.
    
    This preset checks for:
    - Clear abstract summarizing the research
    - Comprehensive literature review with citations
    - Detailed and reproducible methodology
    - Appropriate statistical analysis
    - Discussion acknowledging limitations
    - Properly formatted references
    """
    print("\n" + "="*70)
    print("Example 4: Research Paper Validator (Preset)")
    print("="*70)
    
    # Get the research_paper preset
    preset = get_preset("research_paper")
    
    # Create validator
    validator = DeliverableValidator(Config(), **preset)
    
    print(f"\nValidator configured for academic papers with:")
    print(f"  - {len(validator.validation_criteria)} academic criteria")
    print(f"  - Output format: {validator.output_format}")
    
    # Run validation
    print(f"\nRunning validation...")
    result = await validator.run(SAMPLE_RESEARCH_PAPER)
    
    print(f"\n✓ Validation complete!")
    print(f"\nValidation Result (first 500 chars):")
    print(result['validation'][:500] + "...")
    
    return result

# Run the example
result_4 = await example_4_research_paper()

---

# Part 2: LangGraph Workflows

LangGraph workflows are multi-step agentic systems that orchestrate multiple LLM calls with state management.

## Example 5: ValidationPipeline with Default Prompts

The ValidationPipeline performs validation in two stages:
1. **Quick Scan**: Rapid preliminary check for obvious issues
2. **Detailed Verify**: Thorough validation with specific recommendations

In [ ]:
# Example 5: ValidationPipeline with Default Prompts
async def example_5_default_pipeline():
    """
    Use the ValidationPipeline for multi-step validation.
    
    Workflow:
    1. Quick Scan: Fast preliminary check for format issues, missing sections, obvious errors
    2. Detailed Verify: Thorough validation based on quick scan results
    
    This makes 2 LLM calls and maintains state between them.
    """
    print("\n" + "="*70)
    print("Example 5: ValidationPipeline (Default Prompts)")
    print("="*70)
    
    # Create pipeline with default prompts
    pipeline = ValidationPipeline(config=Config())
    
    print(f"\nPipeline initialized with:")
    print(f"  - 2-step workflow (Quick Scan → Detailed Verify)")
    print(f"  - Default prompts for each step")
    
    # Run the pipeline
    print(f"\nRunning validation pipeline...")
    result = await pipeline.run({"content": SAMPLE_FINANCIAL_REPORT})
    
    print(f"\n✓ Pipeline complete!")
    print(f"\nQuick Scan Result (first 300 chars):")
    print(result['final_result']['quick_scan'][:300] + "...")
    
    print(f"\nDetailed Verification (first 300 chars):")
    print(result['final_result']['detailed_verification'][:300] + "...")
    
    return result

# Run the example
result_5 = await example_5_default_pipeline()

## Example 6: ValidationPipeline with Custom Prompts

Customize the prompts for each step in the pipeline.

In [ ]:
# Example 6: ValidationPipeline with Custom Prompts
async def example_6_custom_pipeline():
    """
    Create a ValidationPipeline with custom prompts for each step.
    
    This allows you to tailor the validation process to your specific needs.
    """
    print("\n" + "="*70)
    print("Example 6: ValidationPipeline (Custom Prompts)")
    print("="*70)
    
    # Define custom prompts for financial compliance
    custom_quick_scan = """Perform a RAPID COMPLIANCE SCAN of this financial deliverable.

Check for:
1. Presence of required sections (Balance Sheet, Income Statement, Cash Flow)
2. Material omissions or obvious errors
3. Format compliance with regulatory standards
4. Critical red flags requiring immediate attention

Content:
{content}

Provide a 2-3 sentence assessment focusing ONLY on critical issues."""

    custom_detailed_verify = """Based on the initial scan, perform a DETAILED COMPLIANCE VERIFICATION.

Initial Scan Results:
{scan_result}

Original Financial Deliverable:
{content}

Provide:
1. Overall Assessment: APPROVED, CONDITIONAL_APPROVAL, or NEEDS_REVISION
2. Compliance Status: For each regulatory requirement
3. Strengths: What was done well
4. Critical Issues: Problems that must be fixed
5. Recommendations: Specific improvements with examples
6. Priority Actions: Ordered list of next steps

Format as JSON."""
    
    # Create pipeline with custom prompts
    pipeline = ValidationPipeline(
        config=Config(),
        quick_scan_prompt=custom_quick_scan,
        detailed_verify_prompt=custom_detailed_verify
    )
    
    print(f"\nPipeline initialized with custom prompts for financial compliance")
    
    # Run the pipeline
    print(f"\nRunning validation pipeline...")
    result = await pipeline.run({"content": SAMPLE_FINANCIAL_REPORT})
    
    print(f"\n✓ Pipeline complete!")
    print(f"\nQuick Scan Result (first 300 chars):")
    print(result['final_result']['quick_scan'][:300] + "...")
    
    print(f"\nDetailed Verification (first 300 chars):")
    print(result['final_result']['detailed_verification'][:300] + "...")
    
    return result

# Run the example
result_6 = await example_6_custom_pipeline()

## Example 7: ValidationPipeline with YAML Configuration

Store pipeline prompts in YAML for easy management.

In [ ]:
# Update the YAML file with pipeline configuration
yaml_config_with_pipeline = """
deliverable_validator:
  system_prompt: |
    You are a financial compliance expert specializing in quarterly reports.
    Focus on accuracy, completeness, and regulatory compliance.
  
  validation_criteria:
    - Executive summary is clear and concise
    - All financial data includes sources
    - Calculations are accurate
    - GAAP compliance is confirmed
    - Risk factors are disclosed
  
  output_format: detailed

validation_pipeline:
  quick_scan_prompt: |
    Perform a QUICK FINANCIAL SCAN of this content.
    
    Look for:
    1. Missing financial statements
    2. Obvious calculation errors
    3. Missing regulatory disclosures
    
    Content:
    {content}
    
    Provide a brief assessment (2-3 sentences).
  
  detailed_verify_prompt: |
    Based on the quick scan, perform DETAILED VERIFICATION.
    
    Quick Scan Results:
    {scan_result}
    
    Original Content:
    {content}
    
    Provide:
    1. Overall assessment (approved/needs_revision)
    2. Specific strengths
    3. Critical issues
    4. Recommendations
    
    Format as JSON.
"""

# Write the updated YAML file
with open('example_pipeline_prompts.yaml', 'w') as f:
    f.write(yaml_config_with_pipeline)

print("Created example_pipeline_prompts.yaml")

In [ ]:
# Example 7: ValidationPipeline with YAML Config
async def example_7_yaml_pipeline():
    """
    Load pipeline configuration from YAML file.
    
    This makes it easy to:
    - Version control your prompts
    - Share configurations across team
    - Update prompts without code changes
    """
    print("\n" + "="*70)
    print("Example 7: ValidationPipeline (YAML Config)")
    print("="*70)
    
    # Create pipeline that loads from YAML
    pipeline = ValidationPipeline(
        config=Config(),
        config_file="example_pipeline_prompts.yaml"
    )
    
    print(f"\nPipeline loaded from YAML with custom prompts")
    
    # Run the pipeline
    print(f"\nRunning validation pipeline...")
    result = await pipeline.run({"content": SAMPLE_FINANCIAL_REPORT})
    
    print(f"\n✓ Pipeline complete!")
    print(f"\nQuick Scan Result (first 300 chars):")
    print(result['final_result']['quick_scan'][:300] + "...")
    
    print(f"\nDetailed Verification (first 300 chars):")
    print(result['final_result']['detailed_verification'][:300] + "...")
    
    return result

# Run the example
result_7 = await example_7_yaml_pipeline()

---

# Summary

## Standalone Agents (Simple Pattern)
- **Use Case**: 80% - Single-purpose validation, batch processing, scheduled jobs
- **Pattern**: Input → LLM call → Output
- **Examples**:
  - Example 1: Preset-based validator (financial_report)
  - Example 2: Programmatic configuration
  - Example 3: YAML configuration
  - Example 4: Different domain preset (research_paper)

## LangGraph Workflows (Complex Pattern)
- **Use Case**: 15% - Multi-step validation, iterative refinement, complex workflows
- **Pattern**: State → Node 1 → Node 2 → ... → Final Result
- **Examples**:
  - Example 5: Default 2-step pipeline
  - Example 6: Custom prompts for each step
  - Example 7: YAML configuration

## Configuration Priority
1. Constructor parameters (highest)
2. YAML config file
3. Environment variables
4. Presets
5. Defaults (lowest)

## Available Presets
- `financial_report` - GAAP/SEC compliance, financial data validation
- `research_paper` - Academic standards, citations, methodology
- `technical_spec` - API documentation, architecture, security
- `marketing_content` - Brand voice, SEO, CTAs
- `legal_document` - Contracts, compliance, clarity
- `general` - General-purpose validation

## Next Steps
1. Copy `.env.development` from agent-workshop repository
2. Add your Claude SDK credentials
3. Choose a preset or create custom configuration
4. Run validation on your content!

## Resources
- [Agent-Workshop GitHub](https://github.com/trentleslie/agent-workshop)
- [Documentation](https://github.com/trentleslie/agent-workshop/blob/main/README.md)
- [Example YAML Config](https://github.com/trentleslie/agent-workshop/blob/main/prompts.yaml.example)